# Mongo vs ClickHouse 

In [1]:
import random as rd

test_comment = " ".join([rd.choice(["lorem", "ipsum", "dolor", "sit", "amet", "consectetur", "adipiscing", "elit", "sed", "do", "eiusmod", "tempor", "incididunt", "ut", "labore", "et", "dolore", "magna", "aliqua"]) for _ in range(50)])
test_comment = test_comment.capitalize() + "."
test_comment = test_comment * 6
test_comment = test_comment[:300]


In [2]:
from datetime import datetime
from uuid import uuid4
from random import choice

users = [str(uuid4()) for _ in range(100)]
films = [str(uuid4()) for _ in range(1000)]

def generate_review_batch()->list[dict]:
    for i in range(10_000):
        print(f"{i} / 10_000")
        batch = [
            {
                "user_id": choice(users),
                "film_id": choice(films),
                "comment": test_comment,
                "timestamp": datetime.now(),
            }
            for _ in range(1000)
        ]
        yield batch

## Mongo

In [3]:
from pymongo import MongoClient

In [4]:
client = MongoClient('localhost', 27019)
db = client['test_database']
collection = db['test_collection']
collection.drop()

In [5]:
import time

start = time.time()
counter = 0
for batch in generate_review_batch():
    collection.insert_many(batch) 
    counter += 1
end = time.time()

print(f"Batch {counter} inserted")
print(f"Insertion took {end - start:.4f} seconds")
print(f"average insertion time: {(end - start) / counter:.4f} seconds")

0 / 10_000
1 / 10_000
2 / 10_000
3 / 10_000
4 / 10_000
5 / 10_000
6 / 10_000
7 / 10_000
8 / 10_000
9 / 10_000
10 / 10_000
11 / 10_000
12 / 10_000
13 / 10_000
14 / 10_000
15 / 10_000
16 / 10_000
17 / 10_000
18 / 10_000
19 / 10_000
20 / 10_000
21 / 10_000
22 / 10_000
23 / 10_000
24 / 10_000
25 / 10_000
26 / 10_000
27 / 10_000
28 / 10_000
29 / 10_000
30 / 10_000
31 / 10_000
32 / 10_000
33 / 10_000
34 / 10_000
35 / 10_000
36 / 10_000
37 / 10_000
38 / 10_000
39 / 10_000
40 / 10_000
41 / 10_000
42 / 10_000
43 / 10_000
44 / 10_000
45 / 10_000
46 / 10_000
47 / 10_000
48 / 10_000
49 / 10_000
50 / 10_000
51 / 10_000
52 / 10_000
53 / 10_000
54 / 10_000
55 / 10_000
56 / 10_000
57 / 10_000
58 / 10_000
59 / 10_000
60 / 10_000
61 / 10_000
62 / 10_000
63 / 10_000
64 / 10_000
65 / 10_000
66 / 10_000
67 / 10_000
68 / 10_000
69 / 10_000
70 / 10_000
71 / 10_000
72 / 10_000
73 / 10_000
74 / 10_000
75 / 10_000
76 / 10_000
77 / 10_000
78 / 10_000
79 / 10_000
80 / 10_000
81 / 10_000
82 / 10_000
83 / 10_000
84

In [ ]:
import timeit

def read_test():
    cursor  = collection.find().batch_size(1000).limit(1000)
    _ = list(cursor)

print(f"Average select time: {timeit.timeit(read_test, number=1000) / 1000} sec")

[{'_id': 'd85ea6bb-e60d-4a5d-93d9-2e5bc2a10b6e', 'count': 100610},
 {'_id': '38b5398a-3f02-472e-8a0d-0ac5e14451a4', 'count': 100537},
 {'_id': '8cefe133-b028-4f8f-a388-ff8ddc6a68df', 'count': 100478},
 {'_id': 'cbfcd7f6-2708-4175-b086-3345316caf3b', 'count': 100477},
 {'_id': '7125b277-1136-4446-b52d-47600cc97721', 'count': 100466},
 {'_id': '1d6e4a98-21fa-40c2-8d8f-5dd30f720e09', 'count': 100456},
 {'_id': '785a5cdc-8034-4e33-9eaf-27d1331f0bdf', 'count': 100423},
 {'_id': '522083c7-3036-4c8b-b2ec-f99f4b19e85d', 'count': 100399},
 {'_id': '9c6ce563-ec27-4f41-aacb-610dccb44233', 'count': 100376},
 {'_id': '98f7172a-81d4-4652-8500-86982ca8e595', 'count': 100353}]

In [9]:
import timeit

pipeline=[
    {
        "$group": {
            "_id": "$user_id",
            "count": {"$sum": 1}
        }
    },
    {
        "$sort": {"count": -1}
    },
    {
        "$limit": 10
    }
]
def aggregate_test():
    cursor = collection.aggregate(pipeline=pipeline)
    list(cursor)

print(f"Average select time: {timeit.timeit(aggregate_test, number=10) / 10} sec")

Average select time: 3.495923020900227 sec


### Insert batch test:
* Batch size = 1000.  10000 batches had been inserted
* Insertion took 169.2879 seconds
* average insertion time: 0.0169 seconds

### Read batch test
* Batch size = 1000.
* Reads Number = 1000 times 
* Average select time: 0.006814309166977182 sec

## Aggregation test
* Reads Number = 1000 times 
* Average aggregate time: 3.495923020900227 sec

## ClickHouse

In [11]:
from clickhouse_driver import Client

In [12]:
client = Client(host="localhost")

In [14]:
client.execute('CREATE DATABASE IF NOT EXISTS collection ON CLUSTER company_cluster;')

[('clickhouse-node1', 9000, 0, '', 3, 0),
 ('clickhouse-node3', 9000, 0, '', 2, 0),
 ('clickhouse-node4', 9000, 0, '', 1, 0),
 ('clickhouse-node2', 9000, 0, '', 0, 0)]

In [15]:
client.execute(
"""CREATE TABLE IF NOT EXISTS collection.test_collection ON CLUSTER company_cluster (
    user_id UUID,
    film_id UUID,
    comment TEXT,
    timestamp TIMESTAMP
)
Engine=MergeTree()
ORDER BY (user_id, film_id, timestamp);""")

[('clickhouse-node1', 9000, 0, '', 3, 0),
 ('clickhouse-node3', 9000, 0, '', 2, 0),
 ('clickhouse-node2', 9000, 0, '', 1, 0),
 ('clickhouse-node4', 9000, 0, '', 0, 0)]

In [17]:
import time

start = time.time()
counter = 0
for batch in generate_review_batch():
    client.execute("INSERT INTO collection.test_collection (user_id, film_id, comment, timestamp) VALUES", batch)
    counter += 1
end = time.time()

print(f"Batch {counter} inserted")
print(f"Insertion took {end - start:.4f} seconds")
print(f"average insertion time: {(end - start) / counter:.4f} seconds")

0 / 10_000
1 / 10_000
2 / 10_000
3 / 10_000
4 / 10_000
5 / 10_000
6 / 10_000
7 / 10_000
8 / 10_000
9 / 10_000
10 / 10_000
11 / 10_000
12 / 10_000
13 / 10_000
14 / 10_000
15 / 10_000
16 / 10_000
17 / 10_000
18 / 10_000
19 / 10_000
20 / 10_000
21 / 10_000
22 / 10_000
23 / 10_000
24 / 10_000
25 / 10_000
26 / 10_000
27 / 10_000
28 / 10_000
29 / 10_000
30 / 10_000
31 / 10_000
32 / 10_000
33 / 10_000
34 / 10_000
35 / 10_000
36 / 10_000
37 / 10_000
38 / 10_000
39 / 10_000
40 / 10_000
41 / 10_000
42 / 10_000
43 / 10_000
44 / 10_000
45 / 10_000
46 / 10_000
47 / 10_000
48 / 10_000
49 / 10_000
50 / 10_000
51 / 10_000
52 / 10_000
53 / 10_000
54 / 10_000
55 / 10_000
56 / 10_000
57 / 10_000
58 / 10_000
59 / 10_000
60 / 10_000
61 / 10_000
62 / 10_000
63 / 10_000
64 / 10_000
65 / 10_000
66 / 10_000
67 / 10_000
68 / 10_000
69 / 10_000
70 / 10_000
71 / 10_000
72 / 10_000
73 / 10_000
74 / 10_000
75 / 10_000
76 / 10_000
77 / 10_000
78 / 10_000
79 / 10_000
80 / 10_000
81 / 10_000
82 / 10_000
83 / 10_000
84

In [18]:
import timeit

def read_test():
    _ = client.execute("SELECT * FROM collection.test_collection LIMIT 1000")

print(f"Average select time: {timeit.timeit(read_test, number=1000) / 1000} sec")


Average select time: 0.012691462625050918 sec


In [21]:
sql_pipeline = """SELECT user_id, COUNT(*) as count FROM collection.test_collection GROUP BY user_id ORDER BY count DESC LIMIT 10 """
def aggregate_test():
    _ = client.execute(sql_pipeline)

print(f"Average select time: {timeit.timeit(aggregate_test, number=10) / 10} sec")

Average select time: 0.17657093749730848 sec


### Insert batch test:
* Batch size = 1000.  10000 batches had been inserted
* Insertion took 233.7237 seconds
* average insertion time: 0.0234 seconds

### Read batch test
* Batch size = 1000.
* Reads Number = 1000 times 
* Average select time: 0.012428301582986023 sec

## Aggregation test
* Reads Number = 1000 times 
* Average select time: 0.17657093749730848 sec